<a id="top"></a>
# Visualization of Roman APT products
***

## Server Information
IMPORTANT: To run this tutorial, please make sure you are logged in the RRN with a medium or large server. Running the parallelized examples in the advanced use cases will require a large server.


## Kernel Information and Read-Only Status

To run this notebook, please select the "Roman Calibration" kernel at the top right of your window.

This notebook is read-only. You can run cells and make edits, but you must save changes to a different location. We recommend saving the notebook within your home directory, or to a new folder within your home (e.g. <span style="font-variant:small-caps;">file > save notebook as > my-nbs/nb.ipynb</span>). Note that a directory must exist before you attempt to add a notebook to it.

## Imports

- *numpy* to handle array functions
- *matplotlib* for plotting data
- *astropy.table* for creating tidy tables of the data
- *pandas* for creating tidy tables of the data
- *pysiaf* to get the coordinates of WFI in different frames of reference (used internally in `footprint_utils`)
- *healpy* to generate all-sky maps (used internally in `footprint_utils`)
- *hpgeom* to handle quick healpix operations (used internally in `footprint_utils`)
- *healsparse* to generate lightweight partial-sky high-resolution HEALPix maps
- *skyproj* to generate sky plots using different projections
- *dask* (Optional) to perform parallel processing

## Introduction

This notebook tries to illustrate several examples of how to visualize APT products in Python. Note that the APT GUI already has Aladdin visualization, and the tool presented in this notebook is complementary.

The utility functions defined in `footprint_utils.py` use the the so-called __simulator input__ file.
* A simulator input file, originally intended to work as input for Roman's image simulation software: `romanIsim`, contains information about all exposures and the position of the `WFI_CEN` aperture, together with duration, and filter element.


On this notebook we will demonstrate how to build visualizations of the simulator input files serially or in parallel. Note: In order to take full advantage of parallelization, the parameters (number of threads / number of processes) need to be optimized according to the machine and inputs used.


Two packages that will be used throughout the notebook are `healsparse` and `skyproj`. If you want to install these, please uncomment the cell below:

In [ ]:
!pip install healpy
!pip install healsparse
!pip install skyproj
!pip install dask[complete]

In [ ]:
import numpy as np
from footprint_utils import *
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import astropy.table

In [ ]:
import healsparse as hsp
import skyproj

By default this notebook will allow you to produce interactive plots, however, some of the plots presented here could lead to reduced performance in certain systems. In order to generate static plots, please the `interactive` variable below to `False`

In [ ]:
interactive = True
if interactive:
    %matplotlib widget
else:
    %matplotlib inline

***

# Creating and loading APT outputs

The [Astronomer's Proposal Tool (APT)](https://www.stsci.edu/scientific-community/software/astronomers-proposal-tool-apt) allows the user to design a General Astrophysics Survey with the Wide Field Instrument, and it was used to design the Roman's Core Community Surveys. Given an APT program, the user can export a variety of products using the command line. In this notebook we will focus on a specific product: the `simulator input` file.

In order to export these files the user can execute the following command in a terminal:

`PATH_TO_APT_PARENT_DIRECTORY/bin/apt -export sim PATH_TO_APT_FILE/my_apt_file.apt -nogui -output PATH_TO_OUTPUT_DIRECTORY`

This command will generate a simulator input file: `my_apt_file.sim.ecsv` in the output directory passed in the `-output` argument. In the next sections we will describe what these files contain and how to read them.

# Working with simulator input files

## Reading a simulator input file `sim.ecsv`

The simulator input file `*.sim.ecsv` is an output from APT intended to be used as input in `romanIsim` to generate simulated images. It contains information about the position `RA, Dec` of the `WFI_CEN` aperture for eaach exposure in the program, together with information about the filter element used, and the duration in seconds (Note: duration is not exposure time, but exposure time + potential slew times / estimated overheads). These files are in `ecsv` format so they can easily be read by `pandas` or `astropy` (recommended for automatic parsing).

In [ ]:
sim_table = astropy.table.Table.read('./aux_data/roman_hlwas.sim.ecsv')

In [ ]:
sim_table

In this case we opted to use `astropy` and got an `astropy.table.Table` object that contains the `RA, DEC, PA` of the `WFI_CEN` aperture. The filter element is displayed in the `BANDAPASS` column. The `MA_TABLE_NUMBER` column indicates the identification number of the MultiAccum table. The `DURATION` column indicates the duration of the exposure including overheads (slews + readout, etc). `PLAN` is the plan number. `PASS` corresponds to `Survey Step` in APT. `SEGMENT` corresponds to the segment number. `OBSERVATION` corresponds to the observation number in a survey step (i.e., the pass number in a survey step and it matches `Observation` in the `obs_all` DataFrame). `VISIT` corresponds to the `Tile` number in the mosaic segment (in the `obs_all` DataFrame). Finally, `EXPOSURE` is the exposure (dither) number for a tile.

So, now, we want to visualize all the exposures in the `F158` filter, and check the total exposure time. So we will take the `sim_table` Table and select all exposures with `BANDPASS == 'F158`. Then we will accumulate the exposure time. The exposures with `MA_TABLE_NUMBER == 5` have an exposure time of 295 seconds, whereas the exposures with `MA_TABLE_NUMBER == 6` have an exposure time of 107 seconds.

In [ ]:
nside_cov = 32  # nside_cov controls the resolution of the low-resolution coverage map (smaller = larger area coverage)
nside_sparse = 4096  # nside_sparse controls the resolution of the high-resolution map (bigger = more resolution)

In [ ]:
table_mask = sim_table['BANDPASS'] == 'F158'
sub_table = sim_table[table_mask]
exp_time = np.ones(np.count_nonzero(table_mask))
exp_time[sub_table['MA_TABLE_NUMBER'] == 6] = 295
exp_time[sub_table['MA_TABLE_NUMBER'] == 5] = 107
ra_sim = sub_table['RA'].data
dec_sim = sub_table['DEC'].data
pa_sim = sub_table['PA'].data

print('Number of selected exposures', len(sub_table))

In this case, instead of a reference position and the relative `V2`, `V3` angles, we directly have the `RA`, `DEC`, and `PA` of the `WFI_CEN` aperture so we can used these directly via `build_single_exp_map_cen`. Let's start with a single segment within the HLWAS medium field (Survey Step 97).

In [ ]:
mask_here = (sub_table['PASS'] == 97) & (sub_table['SEGMENT'] == 1)
print('Going to use', np.count_nonzero(mask_here), 'exposures')
map_result = None
for ra, dec, pa, et in zip(ra_sim[mask_here], dec_sim[mask_here], pa_sim[mask_here], exp_time[mask_here]):
    map_here = build_single_exp_map_cen(ra, dec, pa, et, nside_cov=nside_cov, nside_sparse=nside_sparse)
    if map_result is None:
        map_result = map_here
    else:
        map_result = hsp.sum_union([map_result, map_here])

In [ ]:
fig = plt.figure(4, figsize=(8, 6))
fig.clf()
ax = fig.add_subplot(111)
sp = skyproj.McBrydeSkyproj(ax=ax)

# We define here a custom colorbar for illustration purposes
cmap = plt.cm.Blues  # define the colormap
cmaplist = [cmap(i) for i in range(cmap.N)]
# create the new map
cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0, 400, 100)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
# Note that the default is to zoom in to the range defined by the map
_ = sp.draw_hspmap(map_result, cmap=cmap, norm=norm)
# Set up the colorbar
cbar, _ = sp.draw_inset_colorbar(label='Total exposure time (s)', height='3%', bbox_to_anchor=(-0.06, -0.1, 1, 1), loc=1, ticks=[107, 214, 321])
cbar.ax.set_xticklabels(['107', '214', '321'])
plt.show()

With large programs executing these loops serially becomes cumbersome. Given that each segment is independent, this is a simple-to-parallelize problem. We include an example of how to do so below.

In this case we will use `dask`, and the helper function `partial` in order to compress the arguments of
`build_single_exp_map_cen`. In this case we subdivide the full program into `niter` chunks to prevent using too much memory.

In [ ]:
from dask.distributed import Client, progress, as_completed
import dask.array as da
import dask
from functools import partial
client = Client(processes=True)  # In some instances processes=False will perform better

Now, let's do all of the exposures using the F158 filter element.

Note: For this notebook we cap the number of exposure to 10,000 so the demo runs quickly. Please feel free to remove `nmax` below or set it to -1 in order to run for all `F158` exposures.

Additionally, changing the selection criteria below (`table_mask`) you can choose other interesting subsets of exposures.

In [ ]:
table_mask = sim_table['BANDPASS'] == 'F158'
sub_table = sim_table[table_mask]
exp_time = np.ones(np.count_nonzero(table_mask))
exp_time[sub_table['MA_TABLE_NUMBER'] == 6] = 295
exp_time[sub_table['MA_TABLE_NUMBER'] == 5] = 107

nmax = 10000  # For the example we will just run 10000 remove nmax below or change to -1 in order to run for the full dataset
ra_sim = sub_table['RA'].data[:nmax]
dec_sim = sub_table['DEC'].data[:nmax]
pa_sim = sub_table['PA'].data[:nmax]
exp_time = exp_time[:nmax]

In [ ]:
niter = 3

In [ ]:
map_all = []
for i in range(niter):
    print('Iteration', i)
    imin = (i*len(ra_sim))//niter
    imax = ((i+1)*len(ra_sim))//niter
    ra_subset = ra_sim[imin:imax]
    dec_subset = dec_sim[imin:imax]
    pa_subset = pa_sim[imin:imax]
    exp_subset = exp_time[imin:imax]
    map_here = client.map(partial(build_single_exp_map_cen, nside_sparse=nside_sparse, nside_cov=nside_cov),
                          ra_subset, dec_subset, pa_subset, exp_subset)
    map_partial = client.map(hsp.sum_union, [map_here])
    res_here = map_partial[0].result()
    map_all.append(res_here)


In [ ]:
client.close()  # Close the dask client

In [ ]:
map_f158 = hsp.sum_union(map_all)

In [ ]:
fig = plt.figure(5, figsize=(10, 6))
fig.clf()
ax = fig.add_subplot(111)
sp = skyproj.McBrydeSkyproj(ax=ax)

# We define here a custom colorbar for illustration purposes
cmap = plt.cm.Blues  # define the colormap
cmaplist = [cmap(i) for i in range(cmap.N)]
# create the new map
cmap = matplotlib.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)
bounds = np.linspace(0, 3000, 200)
norm = matplotlib.colors.BoundaryNorm(bounds, cmap.N)
# Note that the default is to zoom in to the range defined by the map
_ = sp.draw_hspmap(map_f158, cmap=cmap, norm=norm)
# Set up the colorbar
cbar, _ = sp.draw_inset_colorbar(label='Total exposure time (s)', height='100%', width='1%', orientation='vertical',
                                 bbox_to_anchor=(+0.06, -0.1, 1, 1), loc=1)
plt.show()

Note that the resulting `healsparse` maps can be saved to disk using the `write` method.

Disclaimer: Depending on the map resolution (controlled by the `nside` parameter) it can generate a very large file.

In [ ]:
# map_f158.write('./data/map_f158_all.hsp')  # Uncomment if you want to write the map to disk. With the default parameters in this notebook it should use 45 MB.

## About this Notebook

**Author(s):** Javier Sánchez <br>
**Keyword(s):** Tutorial, visualization, survey footprints <br>
**Last Updated:** Oct 2025 <br>
**Next Review:** 
***
[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 